# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [98]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_data = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [99]:
from sklearn.model_selection import train_test_split

X = adult_data.drop('income', axis=1)
Y = adult_data['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

1. The random state of the splitting function is a "seed" value that we pass into the `train_test_split` method in order to initialize the pseudo-random number generator. Since this randomization is deterministic (based on the seed passed in), we need to make sure that the "seed" is random enough, so that our results are distinct from other folks who are splitting the same data set. Since we are all using `42` as our seed value, we can expect similar randomization outputs from the `train_test_split` function.

2. This is useful for a few reasons. First, it creates consistency across runs, so that the split is always the same, since it uses the same random seed. This is beneficial for reproducibility. If we didn't set the random state, then the `train-test` split would be different every time, and the results would not be consistent. This would make it difficult to debug issues or compare results with others.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this: 

![](./images/assignment_2__column_transformer.png)

In [100]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

numerical_cols = adult_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = adult_data.select_dtypes(include=['object']).columns.tolist()

num_transformer = Pipeline([
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')), 
    ('scaler', RobustScaler()) 
])


cat_transformer = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')), 
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)) 
])


col_transformer = ColumnTransformer(
    transformers=[
        ('num_transform', num_transformer, numerical_cols), 
        ('cat_transform', cat_transformer, categorical_cols)
    ],
    remainder='drop' 
)



col_transformer.fit(adult_data)

ColumnTransformer(transformers=[('num_transform',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transform',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [101]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

model_pipeline = Pipeline([
    ('preprocessing', col_transformer), 
    ('classifier', RandomForestClassifier(random_state=42))
])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [102]:
from sklearn.model_selection import cross_validate

X = adult_data.drop('income', axis=1)
y = adult_data['income']

cv_results = cross_validate(
    model_pipeline, X, y, cv=5, 
    scoring=['accuracy', 'roc_auc', 'neg_log_loss', 'balanced_accuracy'],
    return_train_score=True
)

print(cv_results)

c:\Users\syednamr\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\syednamr\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


{'fit_time': array([2.66556597, 2.48810697, 2.58105397, 2.43174386, 2.48967028]), 'score_time': array([0.23165727, 0.2718215 , 0.24493766, 0.25024915, 0.23665762]), 'test_accuracy': array([0.85398434, 0.84874079, 0.85273342, 0.85795455, 0.85764742]), 'train_accuracy': array([1.        , 0.99996161, 0.99996161, 1.        , 0.99996161]), 'test_roc_auc': array([0.90294014, 0.89689426, 0.9064731 , 0.90181532, 0.90744798]), 'train_roc_auc': array([1.        , 1.        , 0.99999996, 1.        , 0.99999999]), 'test_neg_log_loss': array([-0.36291439, -0.40442516, -0.34812888, -0.35842469, -0.35055809]), 'train_neg_log_loss': array([-0.07976531, -0.0797981 , -0.08018245, -0.08094565, -0.08093499]), 'test_balanced_accuracy': array([0.77025226, 0.76712445, 0.77889918, 0.7779828 , 0.78017572]), 'train_balanced_accuracy': array([1.        , 0.99997472, 0.99997472, 1.        , 0.99997472])}


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [104]:
import pandas as pd


cv_results = pd.DataFrame(cv_results)
cv_results_sorted = cv_results.sort_values(by='test_neg_log_loss', ascending=True)
print(cv_results_sorted)

   fit_time  score_time  test_accuracy  train_accuracy  test_roc_auc  \
1  2.488107    0.271821       0.848741        0.999962      0.896894   
0  2.665566    0.231657       0.853984        1.000000      0.902940   
3  2.431744    0.250249       0.857955        1.000000      0.901815   
4  2.489670    0.236658       0.857647        0.999962      0.907448   
2  2.581054    0.244938       0.852733        0.999962      0.906473   

   train_roc_auc  test_neg_log_loss  train_neg_log_loss  \
1            1.0          -0.404425           -0.079798   
0            1.0          -0.362914           -0.079765   
3            1.0          -0.358425           -0.080946   
4            1.0          -0.350558           -0.080935   
2            1.0          -0.348129           -0.080182   

   test_balanced_accuracy  train_balanced_accuracy  
1                0.767124                 0.999975  
0                0.770252                 1.000000  
3                0.777983                 1.000000  


Calculate the mean of each metric. 

In [106]:
mean_metrics = cv_results.mean()
print(mean_metrics)

fit_time                   2.531228
score_time                 0.247065
test_accuracy              0.854212
train_accuracy             0.999977
test_roc_auc               0.903114
train_roc_auc              1.000000
test_neg_log_loss         -0.364890
train_neg_log_loss        -0.080325
test_balanced_accuracy     0.774887
train_balanced_accuracy    0.999985
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [117]:
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, balanced_accuracy_score


model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)
y_pred_prob = model_pipeline.predict_proba(X_test)
y_pred_prob_last_col = y_pred_proba[:, 1]

metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'roc_auc': roc_auc_score(y_test, y_pred_prob_last_col),
    'neg_log_loss': log_loss(y_test, y_pred_prob),
    'balanced_accuracy': balanced_accuracy_score(y_test, y_pred)
}


print(metrics)

{'accuracy': 0.8564847988535162, 'roc_auc': 0.9018825137514803, 'neg_log_loss': 0.38733674303408167, 'balanced_accuracy': 0.7784191753807199}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

By converting the text into a binary 1 or 0, it makes it easier to run the RandomForestClassifier and it allows us to make predictions about whether one earns more or less $50k. It is also easy to understand what is happening here, since it is a simple change in the code, so we know what type of data we're working with as we continue making complex transformations, and we don't need to keep track of the state of the `income` as we get to the later stages of development.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.